In [110]:
## 获取URL唯一ID，用于识别唯一舆情
from urllib.parse import urlparse, parse_qs

def getUrlParseId(url):
    if len(url) == 0 or url.isspace():
        return ""
    if url.startswith('‑_ '):
        url = url[3:]
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path
    if path.endswith('/'):
        path = path[0:len(path)-1]
    query_params = parse_qs(parsed_url.query)   
    if "www.douyin.com" == domain:
        splitPath = path.split('/')
        return "DY_" + splitPath[len(splitPath)-1]
    if "www.xiaohongshu.com" == domain or "xhslink.com" == domain:
        splitPath = path.split('/')
        return "XHS_" + splitPath[len(splitPath)-1]
    if "cn.club.vmall.com" == domain:
        splitPath = path.split('/')
        return "HW_" + splitPath[len(splitPath)-1]
    if "weibo.com" == domain:
        splitPath = path.split('/')
        if len(splitPath) == 2:
            return "DY_" + splitPath[len(splitPath)-1]
        return "WB_" + path
    if "web.vip.miui.com" == domain:
        return "XM_" + query_params['postId'][0]
    if "www.xiaomi.cn" == domain:
        splitPath = path.split('/')
        return "XM_" + splitPath[len(splitPath)-1]
    if "www.toutiao.com" == domain:
        splitPath = path.split('/')
        id = splitPath[len(splitPath)-1]
        if id.startswith('i'):
            return "TT_" + id[1:]
        return "TT_" + id
    if "toutiao.com" == domain:
        splitPath = path.split('/')
        return "TT_" + splitPath[len(splitPath)-1]
    if "www.kuaishou.com" == domain:
        splitPath = path.split('/')
        return "KS_" + splitPath[len(splitPath)-1]
    if "bbs.vivo.com.cn" == domain:
        splitPath = path.split('/')
        return "VIVO_" + splitPath[len(splitPath)-1]
    if "www.zhihu.com" == domain:
        splitPath = path.split('/')
        return "ZH_" + splitPath[len(splitPath)-1]
    if "club.hihonor.com" == domain:
        splitPath = path.split('/')
        return "HONOR_" + splitPath[len(splitPath)-1]
    return url

In [111]:
## 爬虫爬取舆情：excel，数据格式：发布日期（2024-06-04），渠道（媒体/论坛/应用商店），媒体（华为+荣耀、小米、今日头条等），舆情URL，舆情内容
spider_file = "/Users/lizhiyang03/Desktop/爬虫-外部舆情.xlsx"

## 人工搜索舆情：excel，数据格式：日期（2024-06-04），类别（媒体/论坛/应用商店），媒体（华为+荣耀、小米、今日头条等），URL，内容
people_file = "/Users/lizhiyang03/Desktop/人工-外部舆情.xlsx"

## 结果输出文件：csv格式，数据格式：日期（2024-06-24），类别，媒体，URL，内容，分类
output_file = "/Users/lizhiyang03/Desktop/最终结果-W22.csv"

In [112]:
## 解析爬虫爬取舆情数据

import pandas as pd
import math
spider_data = {}
df = pd.read_excel(spider_file)
columns = df.columns.values.tolist()
print(columns)
for idx, row in df.iterrows():
    date = row['发布日期']
    category = row['渠道']
    media = row['媒体']
    url = row['舆情URL']
    content = row['舆情内容']

    if type(url) == float:
        url = ''
    if type(content) == float:
        content = ''
    if media == '华为' or media == '荣耀':
        media = '华为+荣耀'
    
    id = content
    if category != '应用商店':
        id = getUrlParseId(url)
    spider_data[id] = ','.join((date, category,media,url,content))
# print(spider_data)

['发布日期', '渠道', '媒体', '舆情URL', '舆情内容', '舆情详细信息']


In [113]:
## 解析人工搜索舆情数据

import pandas as pd
import math
import datetime

people_data = {}
df = pd.read_excel(people_file)
columns = df.columns.values.tolist()
print(columns)
for idx, row in df.iterrows():
    date = row['date']
    category = row['category']
    media = row['media']
    url = row['url']
    content = row['content']
    if type(date) == pd.Timestamp:
        date = date.strftime('%Y-%m-%d')
    
    if type(url) == float:
        url = ''
    if type(content) == float:
        content = ''
    if media == '华为' or media == '荣耀':
        media = '华为+荣耀'
    
    id = content
    if category != '应用商店':
        id = getUrlParseId(url)
    people_data[id] = ','.join((date, category,media,url,content))
# print(people_data)

['category', 'media', 'url', 'content', 'date']


In [114]:
## 判断两个舆情是否重复
result = []
for key,value in people_data.items():
    if key in spider_data:
        result.append(value+',都找到')
    else:
        result.append(value+',人工搜索')
        
for key, value in spider_data.items():
    if key not in people_data:
        result.append(value+',爬虫+人工标注')
print(len(result))
with open(output_file, 'w') as f:
    for item in result:
        f.write(item)
        f.write('\r\n')
    f.flush()

115
